# 🏋️ Fun with Text and Image Embeddings 🍎

Welcome to our **Health & Fitness** embeddings notebook! In this tutorial, we'll show you how to:
1. **Initialize** an `AIProjectClient` to access your Azure AI Foundry project.
2. **Embed text** using `OpenAIClient` with our fun health-themed phrases.
3. **Embed images** using `OpenAIClient` (we're using Cohere's image embeddings model).
4. **Generate a health-themed image** (example code) and display it.
5. **Use a prompt template** for extra context.
Let's get started and have some fun with our healthy ideas! 🍏

> **Disclaimer**: This notebook is for educational purposes only. Always consult a professional for medical advice.

<img src="seq-diagrams/2-embeddings.png">

## 1. Setup & Environment

#### Prerequisites:
- Deploy a [text embeddings model](https://learn.microsoft.com/en-us/azure/ai-foundry/openai/concepts/understand-embeddings) (**text-embedding-3-small**) in Azure AI Foundry
- (Optional) Deploy a image embeddings model (**Cohere-embed-v3-english**) in Azure AI Foundry

#

We'll import our libraries and load the environment variables for:
- `AI_FOUNDRY_PROJECT_ENDPOINT`: Your AI Foundry project endpoint.
- `TEXT_EMBEDDING_MODEL`: The text embeddings model deployment name.
- (Optional) `IMAGE_EMBEDDING_MODEL`: The image embeddings model deployment name.

We'll import libraries, load environment variables, and create an `AIProjectClient`.

> #### Complete [1-basic-chat-completion.ipynb](./1-basic-chat-completion.ipynb) notebook before starting this one

"Let's begin! 🚀"

In [ ]:
#r "nuget: Azure.Identity, 1.18.0-beta.2"
#r "nuget: Azure.AI.Projects, 1.2.0-beta.5"
#r "nuget: dotenv.net"

using System.IO;
using System.ClientModel.Primitives;
using Azure.Identity;
using Azure.AI.Projects;
using Azure.AI.Projects.OpenAI;
using Azure.Core;
using OpenAI;
using OpenAI.Responses;
using dotenv.net;
using OpenAI.Embeddings;

DotEnv.Load(new DotEnvOptions(envFilePaths: new[] { Path.Combine(".","..", ".env") })); 

In [ ]:
#pragma warning disable OPENAI001

var aiFoundryProjectEndpoint = new Uri(Environment.GetEnvironmentVariable("AI_FOUNDRY_PROJECT_ENDPOINT"));
var textEmbeddingModel = Environment.GetEnvironmentVariable("TEXT_EMBEDDING_MODEL");
var imageEmbeddingModel = Environment.GetEnvironmentVariable("IMAGE_EMBEDDING_MODEL");
var tenantId = Environment.GetEnvironmentVariable("TENANT_ID");
Console.WriteLine($"🔑 Using Tenant ID: {tenantId}");

OpenAIClient openAIClient;

try
{
    var credential = new DefaultAzureCredential(new DefaultAzureCredentialOptions
    {
        TenantId = tenantId
    });

    var testToken = credential.GetToken(new TokenRequestContext(new[] { "https://ai.azure.com/.default" }));
    Console.WriteLine("✅ Successfully initialized Azure credentials with correct tenant!");      

    AIProjectClient projectClient = new(aiFoundryProjectEndpoint, credential);
    Console.WriteLine("🎉 Successfully created AIProjectClient");

    var openAiEndpoint = $"https://{projectClient.OpenAI.Endpoint.Authority.Replace(".services.ai.",".openai.")}/openai/v1";
    Console.WriteLine($"Using OpenAI endpoint: {openAiEndpoint}");
    BearerTokenPolicy tokenPolicy = new(
        credential,
        "https://ai.azure.com/.default");

    openAIClient = new(
        authenticationPolicy: tokenPolicy,
        options: new OpenAIClientOptions()
        {
            Endpoint = new(openAiEndpoint),
        }
    );
    Console.WriteLine("🎉 Successfully created OpenAIClient.");
}
catch (Exception ex)
{
    Console.WriteLine("❌ Error initializing client: " + ex.Message);
}

## 2. Text Embeddings

We'll call `GetEmbeddingsClient()` from our `OpenAIClient` to retrieve the embeddings client. Then we'll embed some fun health-themed phrases:
- 🍎 An apple a day keeps the doctor away
- 🏋️ 15-minute HIIT workout routine
- 🧘 Mindful breathing exercises

The output will be numeric vectors representing each phrase in semantic space. Let’s see those embeddings!

In [ ]:
var textPhrases = new List<string>
{
    "An apple a day keeps the doctor away 🍎",
    "15-minute HIIT workout routine 🏋️",
    "Mindful breathing exercises 🧘"
};
EmbeddingClient embeddingClient = null;
try
{
    embeddingClient = openAIClient.GetEmbeddingClient(textEmbeddingModel);
    var response = await embeddingClient.GenerateEmbeddingsAsync(textPhrases);
    foreach(var item in response.Value)
    {
        var vec = item.ToFloats().ToArray();
        var sampleStr = $"[{vec[0]},{vec[1]},...,{vec[vec.Length - 2]}, {vec[vec.Length - 1]}]";
        Console.WriteLine($"Sentence {item.Index}: '{textPhrases[item.Index]}':");
        Console.WriteLine($"Embedding length: {vec.Length}");
        Console.WriteLine($"Sample: {sampleStr}");
    }
}
catch(Exception ex)
{
    Console.WriteLine("❌ Error generating text embeddings: " + ex.Message);
}

# 3. Prompt Template Example 📝

Even though our focus is on embeddings, here's how you might prepend some context to a user message. Imagine you want to embed user text but first add a system prompt such as “You are HealthFitGPT, a fitness guidance model…” This little extra helps set the stage for more context-aware embeddings.

In [ ]:
var TEMPLATE_SYSTEM = @"You are HealthFitGPT, a fitness guidance model.
                        Please focus on healthy advice and disclaim you're not a doctor.
                        User message:"; //We'll append the user message after this.

float[] EmbedWithTemplate(string userMessage)
{
    var fullPrompt = $"{TEMPLATE_SYSTEM} {userMessage}";
    var embeddingResponse = embeddingClient.GenerateEmbeddingsAsync(new List<string> { fullPrompt }).Result;
    return embeddingResponse.Value[0].ToFloats().ToArray();
}

var sampleUserText = "Can you suggest a quick home workout for busy moms?";
var embeddingResult = EmbedWithTemplate(sampleUserText);
Console.WriteLine("Embedding length: " + embeddingResult.Length);
Console.WriteLine("First few dims: " + string.Join(", ", embeddingResult.Take(8)));

# 4. Image Embeddings

Image embeddings typically require managed compute resources. While Azure AI Foundry offers specialized models (like **MedImageInsight**) for medical images, in this example we'll use Cohere's serverless image embedding model.

Here we are using the **`hand-xray.png`** image to generate embeddings. This image (of a hand X-ray) is our fun nod to health-themed imagery!

In [ ]:
try{
    // For image embeddings, we'll use a different approach since Cohere models
    // are typically accessed through the inference client differently
    Console.WriteLine("🔍 Image embeddings require specific model deployment and may not be");
    Console.WriteLine("   available through the standard Azure OpenAI client.");
    Console.WriteLine("   This section demonstrates the concept but may need model-specific configuration.");
    
    // Placeholder for when proper image embedding support is available
    Console.WriteLine("❌ Image embedding functionality needs to be configured with the appropriate");
    Console.WriteLine("   Cohere image embedding model deployment and client setup.");
}   
catch(Exception e){
    Console.WriteLine("❌ Error embedding image:", e);
}


# 5. Generate a Health-Related Image 🏃 (Optional)

>   Note: The example below uses the OpenAI Client with Azure OpenAI's DALL-E 3 model.

Let's generate a health-themed image using OpenAI's DALL-E-3 model. You'll need:

1. A DALL-E-3 model deployment in your Microsoft Foundry resource
2. OpenAIClient configured with your Microsoft Foundry OpenAI endpoint with DefaultAzureCredential authentication.

We'll pass a simple prompt describing a healthy scenario and display the generated image inline.

In [ ]:
#pragma warning disable OPENAI001
using Microsoft.DotNet.Interactive;
using Microsoft.DotNet.Interactive.Formatting;
using static Microsoft.DotNet.Interactive.Formatting.PocketViewTags;

async Task<string> GenerateHealthImage(string prompt="A simple cartoon of a happy person jogging outdoors")
{
    var imageClient = openAIClient.GetImageClient("dall-e-3");
    var imageGenOptions = new OpenAI.Images.ImageGenerationOptions
    {
        Style = OpenAI.Images.GeneratedImageStyle.Natural,
        Quality = OpenAI.Images.GeneratedImageQuality.High,
        Size = OpenAI.Images.GeneratedImageSize.W1024xH1792,
        OutputFileFormat = OpenAI.Images.GeneratedImageFileFormat.Png,
        ResponseFormat = OpenAI.Images.GeneratedImageFormat.Uri
    };
    var imageResponse = await imageClient.GenerateImageAsync(
        prompt: "A watercolor painting of fresh fruits and vegetables arranged in a heart shape",
        imageGenOptions
    );
    var imageUri = imageResponse.Value.ImageUri;
    return imageUri.AbsoluteUri;
}

var imageUrl = await GenerateHealthImage("A watercolor painting of fresh fruits and vegetables arranged in a heart shape");

HTML($"<img src=\"{imageUrl}\" alt=\"Health Image\" style=\"max-width:50%; height:50%;\" />")

# 6. Wrap-Up & Next Steps

🎉 We've shown how to:

- Set up the AIProjectClient & OpenAIClient.
- Get text embeddings using text-embedding-3-small.
- Get image embeddings using Cohere-embed-v3-english on a health-themed (hand X-ray) image.
- Generate a health-themed image (example code).
- Use a prompt template to add system context to your embeddings.

#### Where to go next?

- Explore Evaluation for evaluating your embeddings.
- Use Tracing using OpenTelemetry for end-to-end telemetry.
- Build out a retrieval pipeline to compare similarity of embeddings.

Have fun experimenting, and remember: when it comes to your health, always consult a professional!